In [1]:
# to do : define a function to assess AMD stability as a function of companion properties!
# to do : make sure it always chooses the solution(s) of x that are within 0,1

In [2]:
import pickle
from scipy.optimize import minimize

In [3]:
post_candidate756 = pickle.load( open( "../Radvel_MCMC/CandidateP756/CandidateP756_post_obj.pkl", "rb" ) )
print('P~756 days:')
print(post_candidate756)

post_candidate1632 = pickle.load( open( "../Radvel_MCMC/CandidateP1632/CandidateP1632_post_obj.pkl", "rb" ) )
print('P~1632 days:')
print(post_candidate1632)

P~756 days:
parameter                     value      vary
per1                        31.5786      False
tc1                         13.7689      False
secosw1                    0.581085      False
sesinw1                    0.690174      False
k1                             16.7      False
per2                        808.057       True
tc2                         1379.41       True
secosw2                    0.275457       True
sesinw2                    0.556285       True
k2                          4.89171       True
dvdt                              0      False
curv                              0      False
gamma_j                     1.25338       True
jit_j                       2.88214       True
tp1                         13.5428           
e1                            0.814           
w1                            0.871           
tp2                         1354.71           
e2                          0.38533           
w2                          1.11102           

P

In [4]:
# define variables, all in cgs
# for confirmed inner planet
a = (31.5786/365)**(3/2.)*AU
e = 0.814
m = 48.6*Mearth

# for outer candidate
a_prime = (808.06/356)**(3/2.)*AU
e_prime = 0.39
m_prime = 0.1*m#0.1*m, 2*Mjup

m0 = 1.03*Msun

# define ratios
alpha = a/a_prime
gamma = m/m_prime
mu = G*m0

In [5]:
# define function to compute e_c
def F_abs(ecc):
    """
    returns absolute value of equation 35 in Laskar&Petit2017
    this is so that the minimum will be when the function=0
    """
    term1 = alpha*ecc
    
    top = gamma*ecc
    bottom = np.sqrt(alpha*(1-ecc**2.)+gamma**2.*ecc**2.)
    term2 = top/bottom
    
    return np.abs(term1+term2-1+alpha)

In [6]:
# compute C/Gamma'
curly_C = gamma*np.sqrt(alpha)*(1-np.sqrt(1-e**2.)) + (1-np.sqrt(1-e_prime**2.))

# compute ec, ec'
result = minimize(F_abs,0.5)
e_c = result.x[0] # this is the part that should change!
e_c_prime = 1 - alpha*(1+e_c)

# compute Cc
C_c = gamma*np.sqrt(alpha)*(1-np.sqrt(1-e_c**2.)) + (1-np.sqrt(1-e_c_prime**2.))

In [7]:
# checks
print('e_c results:')
print(result) # 0<e<1, check for multiple x
print('e_c:')
print(e_c) # should be less than 1
print('e_c_prime:')
print(e_c_prime) # should be less than 1

e_c results:
      fun: 0.011050623506180853
 hess_inv: array([[1]])
      jac: array([0.0080366])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 144
      nit: 0
     njev: 70
   status: 2
  success: False
        x: array([0.5])
e_c:
0.5
e_c_prime:
0.9888377729087272


In [8]:
# compute beta
beta = curly_C/C_c
print('beta = ', beta)
if beta<1:
    print('system is AMD stable')
if beta>1:
    print('system is NOT AMD stable')

beta =  0.45598914523556455
system is AMD stable


In [9]:
# 1632d orbit: beta = 0.45 - 0.258
# 808 day orbit: beta = 0.456 - 0.096